In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnull, lit, length, explode, count, upper, lower, regexp_replace, regexp_extract

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data quality check enderecos_clientes ") \
    .config("spark.sql.parquet.enableVectorizedReader", "true") \
    .config("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/10 14:08:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/10 14:08:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.parquet("/app/data/raw/enderecos_clientes.parquet")

25/04/10 14:08:14 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.
25/04/10 14:08:14 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.
25/04/10 14:08:14 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.


In [4]:
df.printSchema()
df.show(10, False)

root
 |-- v_id_cli: string (nullable = true)
 |-- n_sq_end: long (nullable = true)
 |-- d_dt_exc: string (nullable = true)
 |-- v_lcl: string (nullable = true)
 |-- v_uf: string (nullable = true)



+--------------------+--------+---------------------+-----------------+----+
|v_id_cli            |n_sq_end|d_dt_exc             |v_lcl            |v_uf|
+--------------------+--------+---------------------+-----------------+----+
|00D3BFA61A4EB684B1F5|7470026 |NULL                 |CURITIBA         |PR  |
|0313852B5434EC0A76D1|6772409 |NULL                 |LONDRINA         |PR  |
|04194EF6A82A8F1C519C|6459025 |NULL                 |SANTA CRUZ DO SUL|RS  |
|01AF764285A24153070B|5912571 |NULL                 |PORTO ALEGRE     |RS  |
|04A52338FAB5D10204AD|1       |NULL                 |GRAVATAI         |RS  |
|04D139F9FE86539E02F5|1       |NULL                 |CURITIBA         |PR  |
|035A054182C4F2CFF563|1       |NULL                 |CURITIBA         |PR  |
|0489FD8213030BE9A86B|4404117 |2023-03-17 10:42:23.0|PORTO ALEGRE     |RS  |
|04808AA2CCD1D61D82CB|2614487 |NULL                 |PORTO ALEGRE     |RS  |
|025636ACDAC688833A0D|1       |NULL                 |PALHOCA          |SC  |

In [4]:
# Ver o comprimento dos valores
df.select(length(col("n_sq_end")).alias("len")).groupBy("len").count().show()

# Contar valores únicos
df.select("n_sq_end").distinct().count()

+---+------+
|len| count|
+---+------+
|  1| 40800|
|  6|  5474|
|  3|     6|
|  5|   207|
|  4|    18|
|  7|107311|
|  2|   112|
+---+------+



112623

In [8]:
df.select([
    count(when(isnull(c), c)).alias(c) for c in df.columns
]).show()

+--------+--------+--------+-----+----+
|v_id_cli|n_sq_end|d_dt_exc|v_lcl|v_uf|
+--------+--------+--------+-----+----+
|       0|       0|  133819|    0|   0|
+--------+--------+--------+-----+----+



In [5]:
#spark.stop()
